In [1]:
import findspark
import pyspark
import json
import csv
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import uuid

In [2]:
findspark.init()
SparkContext.setSystemProperty('spark.executor.memory', '6g')
conf = SparkConf().setAppName("myFirstApp").setMaster("local")
sc = SparkContext(conf=conf)

C:\Users\abhis\miniconda3\envs\spatial\lib\site-packages\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [3]:
global_file_count = 0
global_record_count = 0
record_reset_threshold = 400001
record_reset_counter = 0

In [4]:
basepath = r"C:\UCR\BigData\Data"
currentFolder = basepath + '\ev'
path_list = []
for root, dirs, files in os.walk(currentFolder):
    for file in files:
        if (file.endswith(".json")):
            path_list.append(os.path.join(root, file))

In [5]:
def get_payload(rawdata):
    payload = {}
    payload['content'] = rawdata['content']
    payload['tweetid'] = rawdata['id']
    return payload

In [6]:
def save_file(tweets):
    filename = 'tweet-ev'.format(uuid.uuid4().hex)
    fullfilepath = '{}\\UpdatedData4L\\{}.json'.format(basepath,filename)
    jsonfile = open(fullfilepath, 'w')
    json.dump(tweets,jsonfile)
    jsonfile.close()
    print(filename)

In [ ]:
tweets = []
count = 0
file_count = 0
for path in path_list:
    textFile = sc.textFile(path)
    global_file_count +=1
    raw_data = textFile.collect()
    global_record_count += len(raw_data)
    for data in raw_data:
        if (record_reset_counter < record_reset_threshold):
            json_object = json.loads(data)
            if (json_object['lang'] == 'en'):
                payload = get_payload(json_object)
                tweets.append(payload)
                record_reset_counter += 1
        else:
            save_file(tweets)
            tweets = []
            record_reset_counter = 0
save_file(tweets)
print('Global File Count {} Global Record Count {}'.format(global_file_count, global_record_count))
print('Done')